In [1]:
import pandas as pd
import numpy as np


In [13]:
#open dataset
df = pd.read_json('dataset.json')
df.head()

,stations,full_timetable
0,"{'Златоуст (1)': ['0', '38', '38', '25', '29',...","{'853': {'route': ['1', '3', '2'], 'free_carri..."
1,"{'Златоуст (1)': ['0', '28', '24', '28', '8', ...","{'896': {'route': ['1', '3', '2'], 'free_carri..."
2,"{'Златоуст (1)': ['0', '25', '15', '6', '15', ...","{'309': {'route': ['1', '3', '2'], 'free_carri..."
3,"{'Златоуст (1)': ['0', '32', '30', '13', '4', ...","{'800': {'route': ['2', '3', '6', '5'], 'free_..."
4,"{'Златоуст (1)': ['0', '10', '38', '16', '1', ...","{'893': {'route': ['1', '3', '6', '5', '7'], '..."


In [14]:
#check the shape of the datafile
df.shape

(100000, 2)

In [103]:
m1 = list(df.iloc[0,0].keys())[0]
m1

'Златоуст (1)'

In [211]:
#смотрим пример данных из первой колонки 
df.iloc[0,0]

{'Златоуст (1)': ['0', '38', '38', '25', '29', '7', '10'],
 'Кыштым (2)': ['26', '0', '7', '34', '20', '27', '35'],
 'Миасс (3)': ['15', '5', '0', '27', '16', '31', '24'],
 'Муслюмово (4)': ['12', '39', '1', '0', '2', '14', '20'],
 'Челябинск (5)': ['38', '38', '1', '28', '0', '33', '14'],
 'Полетаево (6)': ['21', '27', '24', '9', '1', '0', '3'],
 'Еманжелинск (7)': ['9', '3', '23', '25', '32', '37', '0']}

In [87]:
#смотрим пример данных из второй колонки
df.iloc[1,1]

{'896': {'route': ['1', '3', '2'],
  'free_carriage': ['22', '8'],
  'timetable': ['01:00 - 01:25', '03:35 - 04:20', '11:44 - 12:43']},
 '228': {'route': ['1', '3', '2'],
  'free_carriage': ['27', '36'],
  'timetable': ['01:32 - 01:56', '04:06 - 05:06', '12:30 - 13:01']},
 '758': {'route': ['1', '3', '6', '5', '4'],
  'free_carriage': ['17', '28', '2', '36'],
  'timetable': ['02:42 - 04:11',
   '06:21 - 07:51',
   '11:29 - 12:27',
   '13:49 - 14:38',
   '16:48 - 18:06']},
 '346': {'route': ['1', '3', '6', '5', '7'],
  'free_carriage': ['34', '10', '24', '17'],
  'timetable': ['01:59 - 02:14',
   '04:24 - 05:06',
   '08:44 - 09:10',
   '10:32 - 11:24',
   '14:02 - 14:34']},
 '194': {'route': ['2', '3', '1'],
  'free_carriage': ['4', '26'],
  'timetable': ['01:00 - 02:23', '09:47 - 10:14', '12:24 - 13:29']},
 '553': {'route': ['2', '3', '6', '7'],
  'free_carriage': ['20', '19', '2'],
  'timetable': ['02:22 - 03:28',
   '10:52 - 11:46',
   '15:24 - 16:33',
   '17:39 - 19:01']},
 '176': {

In [114]:
#create initial matrix for stations with RC
def station_matrix(k):
    stations = {}
    for i in range(1,8):
        m1 = df.iloc[k,0][list(df.iloc[k,0].keys())[i-1]]
        res = [int(j) for j in m1]
        stations[i]=res
    stations_df = pd.DataFrame.from_dict(stations)
    stations_df.index =[1,2,3,4,5,6,7]
    return stations_df

In [124]:
#create initial empty matrix for stations with potential routes
def station_routes():
    stations = {}
    for i in range(1,8):
        stations[i]=[[],[],[],[],[],[],[]]
    stations_df = pd.DataFrame.from_dict(stations)
    stations_df.index =[1,2,3,4,5,6,7]
    return stations_df

In [125]:
station_routes()

,1,2,3,4,5,6,7
1,[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[]
3,[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[]
5,[],[],[],[],[],[],[]
6,[],[],[],[],[],[],[]
7,[],[],[],[],[],[],[]


In [116]:
n=0
station_matrix(n)

,1,2,3,4,5,6,7
1,0,26,15,12,38,21,9
2,38,0,5,39,38,27,3
3,38,7,0,1,1,24,23
4,25,34,27,0,28,9,25
5,29,20,16,2,0,1,32
6,7,27,31,14,33,0,37
7,10,35,24,20,14,3,0


In [118]:
#get the list of routes from dict:
def get_routes(situation):
    routes_list = []
    for key in situation.keys():
        routes_list.append(situation[key]['route'])
    #print(routes_list)
    return routes_list

In [214]:
#finds unique routes for the specific period
def find_unique_routes(n):
    routes=[]
    situation=df.iloc[n,1]
    routes+=get_routes(situation)
    
    unique = set()
    for my_list in routes:
        unique.add(tuple(my_list))
    unique_routes = list(unique)
    routes_sorted = sorted(unique_routes)
    return routes_sorted

In [207]:
#create sorted by length list of routes 
def find_route(i,j, route_list):
    routes = []
    #print(route_list)
    for route in route_list:
        #print(route)
        try:
            start = route.index(str(i))
            end = route.index(str(j))
            #print(i,j)
        except:
            continue
        r=route[start:end+1]
        if not(r):
            continue
        else:
            routes.append(route[start:end+1])  
    routes = list(set(routes))
    
    routes = [list(item) for item in routes]
    routes.sort(key = len)

    return routes
    

In [216]:
n=0
route_list=find_unique_routes(n)
route_list

[('1', '3', '2'),
 ('1', '3', '6', '5', '4'),
 ('1', '3', '6', '5', '7'),
 ('2', '3', '1'),
 ('2', '3', '6', '5'),
 ('2', '3', '6', '5', '7'),
 ('2', '3', '6', '7'),
 ('3', '2', '4'),
 ('3', '2', '4', '5'),
 ('4', '2'),
 ('4', '2', '3'),
 ('4', '5'),
 ('4', '5', '6', '7'),
 ('4', '5', '7'),
 ('5', '4'),
 ('5', '4', '2', '3', '1'),
 ('5', '6', '3', '1'),
 ('5', '6', '3', '2'),
 ('5', '6', '7'),
 ('5', '7'),
 ('7', '5'),
 ('7', '5', '4'),
 ('7', '5', '6', '3', '1'),
 ('7', '5', '6', '3', '2'),
 ('7', '6', '3', '1'),
 ('7', '6', '5'),
 ('7', '6', '5', '4')]

In [219]:
#fill in the matrix of routes for all stations with sorted list of routes
def fill_route_matrix(n):
    stationroutes = station_routes()
    for i in range(1,8):
        for j in range(1,8):
            if i!=j:
                stationroutes.iloc[i-1,j-1] = find_route(i,j, find_unique_routes(n))
    return stationroutes

In [230]:
n=3
fill_route_matrix(n)

,1,2,3,4,5,6,7
1,[],[],[],[],[],[],[]
2,[],[],"[[2, 3]]","[[2, 4]]","[[2, 4, 5], [2, 3, 6, 5]]","[[2, 3, 6]]",[]
3,"[[3, 1]]","[[3, 2]]",[],"[[3, 2, 4]]","[[3, 6, 5], [3, 2, 4, 5]]","[[3, 6]]",[]
4,"[[4, 5, 6, 3, 1]]","[[4, 2]]","[[4, 2, 3], [4, 5, 6, 3]]",[],"[[4, 5]]","[[4, 5, 6]]",[]
5,"[[5, 6, 3, 1]]","[[5, 6, 3, 2]]","[[5, 6, 3]]","[[5, 4]]",[],"[[5, 6]]",[]
6,"[[6, 3, 1]]","[[6, 3, 2]]","[[6, 3]]",[],"[[6, 5]]",[],[]
7,[],"[[7, 6, 3, 2], [7, 5, 6, 3, 2]]","[[7, 6, 3], [7, 5, 6, 3]]",[],"[[7, 5], [7, 6, 5]]","[[7, 6], [7, 5, 6]]",[]


In [228]:
output = {'Станция отправления': ['1'], 'Станция конечная': ['7'], 'Станция промежуточная': ['5'],'Число вагонов': [10], '№ маршрута': ['803'], 'Время отправки': ['01:30'],'Время прибытия': ['06:00']} 
output = pd.DataFrame.from_dict(output)

In [229]:
output

,Станция отправления,Станция конечная,Станция промежуточная,Число вагонов,№ маршрута,Время отправки,Время прибытия
0,1,7,5,10,803,01:30,06:00
